In [42]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import os, sys
# add parent directory to path
sys.path.append("..")

from model import TimeSeriesDataset, TimeSeriesTransformer, TimeSeriesTransformerRotary, WorldModel

%load_ext autoreload
%autoreload 2





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:

with open('/abiomed/downsampled/10min_1hr_all_data.pkl', 'rb') as f:
    data = pickle.load(f)

print(data.keys())



dict_keys(['train', 'val', 'test', 'mean', 'std'])


# Transformer

In [ ]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

# Load data
data_path = '/abiomed/downsampled/10min_1hr_all_data.pkl'
print(f"Loading data from: {data_path}")

with open(data_path, 'rb') as f:
    data = pickle.load(f)

print("Data keys:", data.keys())
print(f"Train data shape: {data['train'].shape}")
print(f"Val data shape: {data['val'].shape}")
print(f"Test data shape: {data['test'].shape}")
print(f"Number of features: {data['train'].shape[2]}")

# Model parameters
num_features = 12  # Number of input features
input_horizon = 6
forecast_horizon = 6  # Number of time steps to forecast
output_dim = (num_features - 1) * forecast_horizon  # Exclude p-level from output
    
# Create WorldModel with  transformer
print("\nCreating WorldModel with transformer...")
wm = WorldModel(
    num_features=num_features,
    dim_model=256,
    num_heads=8,
    num_encoder_layers=3,
    num_decoder_layers=2,
    encoder_dropout=0.1,
    decoder_dropout=0.0,
    max_len=100,
    forecast_horizon=forecast_horizon,
    model_type='transformer',    
    device=device
)
    
print(f"Model created with {num_features} input features")
print(f"Input horizon: {input_horizon}, Forecast horizon: {forecast_horizon}")

# Load data into the model
print("\nLoading data into the model...")
wm.load_data(data_path)
print("Data loaded successfully!")

# Training parameters
num_epochs = 50
batch_size = 64
learning_rate = 0.001

print(f"\nStarting training with:")
print(f"Epochs: {num_epochs}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")

# Train the model
print("\nTraining the model...")
best_model = wm.train_model(
    num_epochs=num_epochs,
    batch_size=batch_size,
    learning_rate=learning_rate
)

print("\nTraining completed!")


# Rotary

In [43]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

# Load data
data_path = '/abiomed/downsampled/10min_1hr_all_data.pkl'
print(f"Loading data from: {data_path}")

with open(data_path, 'rb') as f:
    data = pickle.load(f)

print("Data keys:", data.keys())
print(f"Train data shape: {data['train'].shape}")
print(f"Val data shape: {data['val'].shape}")
print(f"Test data shape: {data['test'].shape}")
print(f"Number of features: {data['train'].shape[2]}")

# Model parameters
num_features = 12  # Number of input features
input_horizon = 6
forecast_horizon = 6  # Number of time steps to forecast
output_dim = (num_features - 1) * forecast_horizon  # Exclude p-level from output
    
# Create WorldModel with rotary transformer
print("\nCreating WorldModel with rotary transformer...")
wm = WorldModel(
    num_features=num_features,
    dim_model=256,
    num_heads=8,
    num_encoder_layers=3,
    num_decoder_layers=2,
    encoder_dropout=0.1,
    decoder_dropout=0.0,
    max_len=100,
    forecast_horizon=forecast_horizon,
    model_type='rotary_transformer',  # Use rotary transformer
    device=device
)
    
print(f"Model created with {num_features} input features")
print(f"Input horizon: {input_horizon}, Forecast horizon: {forecast_horizon}")

# Load data into the model
print("\nLoading data into the model...")
wm.load_data(data_path)
print("Data loaded successfully!")

# Training parameters
num_epochs = 50
batch_size = 64
learning_rate = 0.001

print(f"\nStarting training with:")
print(f"Epochs: {num_epochs}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")

# Train the model
print("\nTraining the model...")
best_model = wm.train_model(
    num_epochs=num_epochs,
    batch_size=batch_size,
    learning_rate=learning_rate
)

print("\nTraining completed!")


Loading data from: /abiomed/downsampled/10min_1hr_all_data.pkl
Data keys: dict_keys(['train', 'val', 'test', 'mean', 'std'])
Train data shape: torch.Size([12051, 12, 13])
Val data shape: torch.Size([1938, 12, 13])
Test data shape: torch.Size([3876, 12, 13])
Number of features: 13

Creating WorldModel with rotary transformer...
time series transformer device cuda:1
Model created with 12 input features
Input horizon: 6, Forecast horizon: 6

Loading data into the model...
loaded datasets with length 
 train:  12051 
 val:  1938 
 test:  3876
Data loaded successfully!

Starting training with:
Epochs: 50
Batch size: 64
Learning rate: 0.001

Training the model...
New best model with val loss: 0.1503
Epoch 1/50 | Train Loss: 0.1753 | Val Loss: 0.1503 | Val MAPE: 0.2734
Final test mse: 0.1432153880596161
Final test MAP mse: 2.862
New best model with val loss: 0.1203
Epoch 2/50 | Train Loss: 0.1256 | Val Loss: 0.1203 | Val MAPE: 0.2280
New best model with val loss: 0.1146
Epoch 3/50 | Train Los

In [44]:

# Test the trained model
print("\nTesting the trained model...")
test_mse, test_mape = wm.test(loss_fn='mae')
print(f"Final Test MAE: {test_mse:.6f}")
print(f"Final Test MAPE: {test_mape:.3f}")


Testing the trained model...
Final test mae: 0.20064054429531097
Final test MAP mae: 3.835
Final Test MAE: 0.200641
Final Test MAPE: 3.835


In [45]:
# Save the trained model
model_save_path = "/abiomed/downsampled/models/rotary_1hr_mse3.pth"
print(f"\nSaving model to: {model_save_path}")
wm.save_model(model_save_path)
print(f"Model saved successfully!")


Saving model to: /abiomed/downsampled/models/rotary_1hr_mse3.pth
Model saved successfully!


In [39]:

# Create WorldModel with rotary transformer
print("\nCreating WorldModel with rotary transformer...")
wm_mae = WorldModel(
    num_features=num_features,
    dim_model=256,
    num_heads=8,
    num_encoder_layers=3,
    num_decoder_layers=2,
    encoder_dropout=0.1,
    decoder_dropout=0.0,
    max_len=100,
    forecast_horizon=forecast_horizon,
    model_type='rotary_transformer',  # Use rotary transformer
    device=device
)
    
print(f"Model created with {num_features} input features")
print(f"Input horizon: {input_horizon}, Forecast horizon: {forecast_horizon}")

# Load data into the model
print("\nLoading data into the model...")
wm_mae.load_data(data_path)
print("Data loaded successfully!")

# Training parameters
num_epochs = 100
batch_size = 64
learning_rate = 0.001

print(f"\nStarting training with:")
print(f"Epochs: {num_epochs}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")

# Train the model
print("\nTraining the model...")
best_model = wm_mae.train_model(
    num_epochs=num_epochs,
    batch_size=batch_size,
    learning_rate=learning_rate,
    loss_fn='mae'
)

print("\nTraining completed!")

# Test the trained model
print("\nTesting the trained model...")
test_mse, test_mape = wm_mae.test(loss_fn='mae')
print(f"Final Test MAE: {test_mse:.6f}")
print(f"Final Test MAP MAE: {test_mape:.3f}")



Creating WorldModel with rotary transformer...
time series transformer device cuda:1
Model created with 12 input features
Input horizon: 6, Forecast horizon: 6

Loading data into the model...
loaded datasets with length 
 train:  12051 
 val:  1938 
 test:  3876
Data loaded successfully!

Starting training with:
Epochs: 100
Batch size: 64
Learning rate: 0.001

Training the model...
New best model with val loss: 0.2434
Epoch 1/100 | Train Loss: 0.2685 | Val Loss: 0.2434 | Val MAPE: 0.3711
Final test mae: 0.24355734884738922
Final test MAP mae: 4.306
New best model with val loss: 0.2197
Epoch 2/100 | Train Loss: 0.2082 | Val Loss: 0.2197 | Val MAPE: 0.3470
New best model with val loss: 0.2013
Epoch 3/100 | Train Loss: 0.1964 | Val Loss: 0.2013 | Val MAPE: 0.3200
New best model with val loss: 0.2008
Epoch 4/100 | Train Loss: 0.1891 | Val Loss: 0.2008 | Val MAPE: 0.3203
Epoch 5/100 | Train Loss: 0.1882 | Val Loss: 0.2055 | Val MAPE: 0.3537
New best model with val loss: 0.1956
Epoch 6/100 

In [40]:

# Test the trained model
print("\nTesting the trained model...")
test_mse, test_mape = wm.test(loss_fn='mae')
print(f"Final Test MAE: {test_mse:.6f}")
print(f"Final Test MAP MAE: {test_mape:.3f}")


Testing the trained model...
Final test mae: 0.20836730301380157
Final test MAP mae: 3.856
Final Test MAE: 0.208367
Final Test MAP MAE: 3.856


In [41]:
# Save the trained model
model_save_path = "/abiomed/downsampled/models/rotary_1hr_mae.pth"
print(f"\nSaving model to: {model_save_path}")
wm.save_model(model_save_path)
print(f"Model saved successfully!")


Saving model to: /abiomed/downsampled/models/rotary_1hr_mae.pth
Model saved successfully!


In [24]:
mv /abiomed/downsampled/models/10min_1hr_rotary_model.pth /abiomed/downsampled/models/rotary_1hr_mse2.pth

In [25]:
ls /abiomed/downsampled/models/

10min_1hr_window_9feat_model.pth   5min_1hr_window_model.pth
10min_1hr_window_9feat_model2.pth  5min_2hr_window_model.pth
10min_1hr_window_model.pth         rotary_1hr_mse.pth
10min_2hr_window_model.pth         rotary_1hr_mse2.pth
